In [1]:
import pandas as pd
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import json
from tqdm import tqdm
from rdkit import Chem
from torch_geometric.loader import DataLoader
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# Load the CSV file
file_path = '파일경로'
data = pd.read_csv(file_path)


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [68]:
#gpu 메모리 리셋

# from numba import cuda 
# device = cuda.get_current_device()
# device.reset() -> 쓰면 쿠다 오류남

import gc

# GPU 메모리 해제
torch.cuda.empty_cache()
gc.collect()

0

In [4]:
data

,Unnamed: 0,PUBCHEM_EXT_DATASOURCE_SMILES,Sequence,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE
0,0,C1=CC=C(C=C1)CCC(=O)NC(=S)NC2=CC=CC=C2[N+](=O)...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0
1,1,CC1=C(C=C(C=C1)NS(=O)(=O)C2=CC3=C(C=C2)NC(=O)C...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0
2,2,CC1=C(C=CC(=C1)F)S(=O)(=O)NCC(C2=CN=CC=C2)N3CC...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0
3,3,C1CCC(=NNC(=O)CN2C3C(NC(=O)N3)NC2=O)C1,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0
4,4,C1=CC(=CC(=C1)C(F)(F)F)CNC(=O)CSCC(=O)O,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0
...,...,...,...,...,...
624299,927585,CC(=O)N=C(N)SC1=NC2=CC=CC=C2N1,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0
624300,927586,COC1=C(C2=C(C=C1)C=NN(C2=O)C3=CC=C(C=C3)OC(F)(...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0
624301,927587,CC1=CC=C(C=C1)C(=O)N/C(=C\C2=CC(=CC=C2)[N+](=O...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0
624302,927588,COC1=CC=C(C=C1)/C=C/C(=O)NCCCN2C=CN=C2,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0


In [5]:
data[['PUBCHEM_ACTIVITY_OUTCOME']].value_counts()

PUBCHEM_ACTIVITY_OUTCOME
Inactive                    616117
Inconclusive                  6538
Active                        1649
Name: count, dtype: int64

In [6]:
#아미노산 극성

amino_acid_polarity = {
    'A': 0, 'R': 1, 'N': 1, 'D': 1, 'C': 0, 'E': 1, 'Q': 1, 'G': 0,
    'H': 1, 'I': 0, 'L': 0, 'K': 1, 'M': 0, 'F': 0, 'P': 0, 'S': 1,
    'T': 1, 'W': 0, 'Y': 1, 'V': 0
}

In [7]:

# Function to convert a protein sequence to a graph using NetworkX
def sequence_to_graph(sequence):
    G = nx.Graph()
    for i, aa in enumerate(sequence):
        polarity = amino_acid_polarity.get(aa, 0)
        G.add_node(i, amino_acid=aa, polarity=polarity)
        if i > 0:
            G.add_edge(i-1, i)
    return G


# Convert the two unique sequences to graphs
unique_sequences = data['Sequence'].unique()[:2]  # Assume there are only two unique sequences
protein_graphs = {seq: sequence_to_graph(seq) for seq in unique_sequences}

# Map the graphs back to the dataframe
data['ProteinGraph'] = data['Sequence'].map(protein_graphs)


In [8]:
data

,Unnamed: 0,PUBCHEM_EXT_DATASOURCE_SMILES,Sequence,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,ProteinGraph
0,0,C1=CC=C(C=C1)CCC(=O)NC(=S)NC2=CC=CC=C2[N+](=O)...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,1,CC1=C(C=C(C=C1)NS(=O)(=O)C2=CC3=C(C=C2)NC(=O)C...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,2,CC1=C(C=CC(=C1)F)S(=O)(=O)NCC(C2=CN=CC=C2)N3CC...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,3,C1CCC(=NNC(=O)CN2C3C(NC(=O)N3)NC2=O)C1,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
4,4,C1=CC(=CC(=C1)C(F)(F)F)CNC(=O)CSCC(=O)O,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
...,...,...,...,...,...,...
624299,927585,CC(=O)N=C(N)SC1=NC2=CC=CC=C2N1,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
624300,927586,COC1=C(C2=C(C=C1)C=NN(C2=O)C3=CC=C(C=C3)OC(F)(...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
624301,927587,CC1=CC=C(C=C1)C(=O)N/C(=C\C2=CC(=CC=C2)[N+](=O...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
624302,927588,COC1=CC=C(C=C1)/C=C/C(=O)NCCCN2C=CN=C2,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


In [9]:
def graph_to_data(G):
    x = torch.tensor([G.nodes[i]['polarity'] for i in G.nodes], dtype=torch.float).view(-1, 1)
    edge_index = torch.tensor(list(G.edges)).t().contiguous()
    return Data(x=x, edge_index=edge_index)

In [10]:
# 그래프를 Data 형식으로 변환
protein_data = {seq: graph_to_data(protein_graphs[seq]) for seq in unique_sequences}


# DataFrame에 Data 객체 맵핑
data['ProteinGraph'] = data['Sequence'].map(protein_data)

In [11]:
data

,Unnamed: 0,PUBCHEM_EXT_DATASOURCE_SMILES,Sequence,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,ProteinGraph
0,0,C1=CC=C(C=C1)CCC(=O)NC(=S)NC2=CC=CC=C2[N+](=O)...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])..."
1,1,CC1=C(C=C(C=C1)NS(=O)(=O)C2=CC3=C(C=C2)NC(=O)C...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])..."
2,2,CC1=C(C=CC(=C1)F)S(=O)(=O)NCC(C2=CN=CC=C2)N3CC...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])..."
3,3,C1CCC(=NNC(=O)CN2C3C(NC(=O)N3)NC2=O)C1,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])..."
4,4,C1=CC(=CC(=C1)C(F)(F)F)CNC(=O)CSCC(=O)O,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])..."
...,...,...,...,...,...,...
624299,927585,CC(=O)N=C(N)SC1=NC2=CC=CC=C2N1,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])..."
624300,927586,COC1=C(C2=C(C=C1)C=NN(C2=O)C3=CC=C(C=C3)OC(F)(...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])..."
624301,927587,CC1=CC=C(C=C1)C(=O)N/C(=C\C2=CC(=CC=C2)[N+](=O...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])..."
624302,927588,COC1=CC=C(C=C1)/C=C/C(=O)NCCCN2C=CN=C2,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])..."


In [12]:

# Convert SMILES to molecular graphs using RDKit
def mol_to_graph(mol):
    atom_features = []
    edge_index = []
    for atom in mol.GetAtoms():
        atom_features.append([atom.GetAtomicNum()])
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.append([i, j])
        edge_index.append([j, i])
    x = torch.tensor(atom_features, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    return Data(x=x, edge_index=edge_index)

def smiles_to_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol_to_graph(mol)

# Convert SMILES to molecular graphs
data['MoleculeGraph'] = data['PUBCHEM_EXT_DATASOURCE_SMILES'].apply(smiles_to_mol)


[16:28:58] WARNING: not removing hydrogen atom without neighbors
[16:29:00] WARNING: not removing hydrogen atom without neighbors
[16:30:45] WARNING: not removing hydrogen atom without neighbors
[16:30:47] WARNING: not removing hydrogen atom without neighbors


In [13]:
data

,Unnamed: 0,PUBCHEM_EXT_DATASOURCE_SMILES,Sequence,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,ProteinGraph,MoleculeGraph
0,0,C1=CC=C(C=C1)CCC(=O)NC(=S)NC2=CC=CC=C2[N+](=O)...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])..."
1,1,CC1=C(C=C(C=C1)NS(=O)(=O)C2=CC3=C(C=C2)NC(=O)C...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])..."
2,2,CC1=C(C=CC(=C1)F)S(=O)(=O)NCC(C2=CN=CC=C2)N3CC...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])..."
3,3,C1CCC(=NNC(=O)CN2C3C(NC(=O)N3)NC2=O)C1,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])..."
4,4,C1=CC(=CC(=C1)C(F)(F)F)CNC(=O)CSCC(=O)O,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])..."
...,...,...,...,...,...,...,...
624299,927585,CC(=O)N=C(N)SC1=NC2=CC=CC=C2N1,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([6.]), tensor([8.])..."
624300,927586,COC1=C(C2=C(C=C1)C=NN(C2=O)C3=CC=C(C=C3)OC(F)(...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([8.]), tensor([6.])..."
624301,927587,CC1=CC=C(C=C1)C(=O)N/C(=C\C2=CC(=CC=C2)[N+](=O...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])..."
624302,927588,COC1=CC=C(C=C1)/C=C/C(=O)NCCCN2C=CN=C2,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inactive,0.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([8.]), tensor([6.])..."


In [14]:
label_mapping = {'Inactive': 0, 'Active': 1, 'Inconclusive': 2}
data['ActivityLabel'] = data['PUBCHEM_ACTIVITY_OUTCOME'].map(label_mapping)

In [15]:
from sklearn.utils import resample

# 클래스별 샘플 수 계산
inactive = data[data['ActivityLabel'] == 0]
active = data[data['ActivityLabel'] == 1]
inconclusive = data[data['ActivityLabel'] == 2]

# 단백질 유형 분류
protein2in = inactive[inactive['Sequence'].str.contains('MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKYSSRTEFRDKTDYIMYNPRPRDEPSSENPVSVSPLLCELAAARSRIHFNPTETTIGIVTCGGICPGLNDVIRSITLTGINVYNVKRVIGFRFGYWGLSKKGSQTAIELHRGRVTNIHHYGGTILGSSRGPQDPKEMVDTLERLGVNILFTVGGDGTQRGALVISQEAKRRGVDISVFGVPKTIDNDLSFSHRTFGFQTAVEKAVQAIRAAYAEAVSANYGVGVVKLMGRDSGFIAAQAAVASAQANICLVPENPISEQEVMSLLERRFCHSRSCVIIVAEGFGQDWGRGSGGYDASGNKKLIDIGVILTEKVKAFLKANKSRYPDSTVKYIDPSYMIRACPPSANDALFCATLATLAVHEAMAGATGCIIAMRHNNYILVPIKVATSVRRVLDLRGQLWRQVREITVDLGSDVRLARKLEIRRELEAINRNRDRLHEELAKL')]
protein1in = inactive[inactive['Sequence'].str.contains('MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGMSVARMNFSHGSHEYHQTTINNVRQAAAELGVNIAIALDTKGPEIRTGQFVGGDAVMERGATCYVTTDPAFADKGTKDKFYIDYQNLSKVVRPGNYIYIDDGILILQVQSHEDEQTLECTVTNSHTISDRRGVNLPGCDVDLPAVSAKDRVDLQFGVEQGVDMIFASFIRSAEQVGDVRKALGPKGRDIMIICKIENHQGVQNIDSIIEESDGIMVARGDLGVEIPAEKVVVAQKILISKCNVAGKPVICATQMLESMTYNPRPTRAEVSDVANAVFNGADCVMLSGETAKGKYPNEVVQYMARICLEAQSALNEYVFFNSIKKLQHIPMSADEAVCSSAVNSVYETKAKAMVVLSNTGRSARLVAKYRPNCPIVCVTTRLQTCRQLNITQGVESVFFDADKLGHDEGKEHRVAAGVEFAKSKGYVQTGDYCVVIHADHKVKGYANQTRILLVE')]

protein2a = active[active['Sequence'].str.contains('MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKYSSRTEFRDKTDYIMYNPRPRDEPSSENPVSVSPLLCELAAARSRIHFNPTETTIGIVTCGGICPGLNDVIRSITLTGINVYNVKRVIGFRFGYWGLSKKGSQTAIELHRGRVTNIHHYGGTILGSSRGPQDPKEMVDTLERLGVNILFTVGGDGTQRGALVISQEAKRRGVDISVFGVPKTIDNDLSFSHRTFGFQTAVEKAVQAIRAAYAEAVSANYGVGVVKLMGRDSGFIAAQAAVASAQANICLVPENPISEQEVMSLLERRFCHSRSCVIIVAEGFGQDWGRGSGGYDASGNKKLIDIGVILTEKVKAFLKANKSRYPDSTVKYIDPSYMIRACPPSANDALFCATLATLAVHEAMAGATGCIIAMRHNNYILVPIKVATSVRRVLDLRGQLWRQVREITVDLGSDVRLARKLEIRRELEAINRNRDRLHEELAKL')]
protein1a = active[active['Sequence'].str.contains('MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGMSVARMNFSHGSHEYHQTTINNVRQAAAELGVNIAIALDTKGPEIRTGQFVGGDAVMERGATCYVTTDPAFADKGTKDKFYIDYQNLSKVVRPGNYIYIDDGILILQVQSHEDEQTLECTVTNSHTISDRRGVNLPGCDVDLPAVSAKDRVDLQFGVEQGVDMIFASFIRSAEQVGDVRKALGPKGRDIMIICKIENHQGVQNIDSIIEESDGIMVARGDLGVEIPAEKVVVAQKILISKCNVAGKPVICATQMLESMTYNPRPTRAEVSDVANAVFNGADCVMLSGETAKGKYPNEVVQYMARICLEAQSALNEYVFFNSIKKLQHIPMSADEAVCSSAVNSVYETKAKAMVVLSNTGRSARLVAKYRPNCPIVCVTTRLQTCRQLNITQGVESVFFDADKLGHDEGKEHRVAAGVEFAKSKGYVQTGDYCVVIHADHKVKGYANQTRILLVE')]

protein2c = inconclusive[inconclusive['Sequence'].str.contains('MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKYSSRTEFRDKTDYIMYNPRPRDEPSSENPVSVSPLLCELAAARSRIHFNPTETTIGIVTCGGICPGLNDVIRSITLTGINVYNVKRVIGFRFGYWGLSKKGSQTAIELHRGRVTNIHHYGGTILGSSRGPQDPKEMVDTLERLGVNILFTVGGDGTQRGALVISQEAKRRGVDISVFGVPKTIDNDLSFSHRTFGFQTAVEKAVQAIRAAYAEAVSANYGVGVVKLMGRDSGFIAAQAAVASAQANICLVPENPISEQEVMSLLERRFCHSRSCVIIVAEGFGQDWGRGSGGYDASGNKKLIDIGVILTEKVKAFLKANKSRYPDSTVKYIDPSYMIRACPPSANDALFCATLATLAVHEAMAGATGCIIAMRHNNYILVPIKVATSVRRVLDLRGQLWRQVREITVDLGSDVRLARKLEIRRELEAINRNRDRLHEELAKL')]
protein1c = inconclusive[inconclusive['Sequence'].str.contains('MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGMSVARMNFSHGSHEYHQTTINNVRQAAAELGVNIAIALDTKGPEIRTGQFVGGDAVMERGATCYVTTDPAFADKGTKDKFYIDYQNLSKVVRPGNYIYIDDGILILQVQSHEDEQTLECTVTNSHTISDRRGVNLPGCDVDLPAVSAKDRVDLQFGVEQGVDMIFASFIRSAEQVGDVRKALGPKGRDIMIICKIENHQGVQNIDSIIEESDGIMVARGDLGVEIPAEKVVVAQKILISKCNVAGKPVICATQMLESMTYNPRPTRAEVSDVANAVFNGADCVMLSGETAKGKYPNEVVQYMARICLEAQSALNEYVFFNSIKKLQHIPMSADEAVCSSAVNSVYETKAKAMVVLSNTGRSARLVAKYRPNCPIVCVTTRLQTCRQLNITQGVESVFFDADKLGHDEGKEHRVAAGVEFAKSKGYVQTGDYCVVIHADHKVKGYANQTRILLVE')]



In [16]:
print(len(active),len(inconclusive),len(protein2a),len(protein1a),len(protein1c),len(protein2c))

1649 6538 560 1089 2003 4535


In [17]:

# 언더샘플링
underprotein1 = resample(protein1in, replace=False, n_samples= (len(active) + len(inconclusive))//2, random_state=42)
underprotein2 = resample(protein2in, replace=False, n_samples= (len(active) + len(inconclusive))//2, random_state=42)

# underprotein1 = resample(protein1in, replace=False, n_samples= (len(active) + len(inconclusive))//2 + 221 , random_state=42)
# underprotein2 = resample(protein2in, replace=False, n_samples= (len(active) + len(inconclusive))//2 + 221 , random_state=42)

# 오버샘플링
# overprotein1a = resample(protein1a, replace=True, n_samples=3004 + 221, random_state=42)
# overprotein2a = resample(protein2a, replace=True, n_samples=3533 + 221, random_state=42)
# overprotein1c = resample(protein1c, replace=True, n_samples=2090, random_state=42)
# overprotein2c = resample(protein2c, replace=True, n_samples=len(underprotein2), random_state=42)

# # 다른 클래스 샘플 유지
inconclusive_samples = data[data['PUBCHEM_ACTIVITY_OUTCOME'] == 'Inconclusive']
active_samples = data[data['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']

In [18]:
# 데이터 결합
# balanced_data = pd.concat([underprotein1,underprotein2, protein1a, overprotein1a, protein2a, overprotein2a, protein1c, overprotein1c, protein2c])
balanced_data = pd.concat([underprotein1,underprotein2,active_samples,inconclusive_samples])

In [19]:
balanced_data

,Unnamed: 0,PUBCHEM_EXT_DATASOURCE_SMILES,Sequence,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,ProteinGraph,MoleculeGraph,ActivityLabel
152188,152188,CN1C(=CC(=NS1(=O)=O)C2=CC=CS2)C(=O)NC3=CC=C(C=...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([7.]), tensor([6.])...",0
70469,70469,CC(=O)NC(CCS(=O)(=O)C)C(=O)NC1=CC=CC=C1C(=O)OC,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([8.])...",0
137845,137845,CN1C(=C(C(=N1)C(F)(F)F)C2=CC=CC=C2)NC(=O)CSCC(...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([7.]), tensor([6.])...",0
56640,56640,CC1=CC=CC=C1N(C(=O)C)/N=C\2/C(=O)C3=CC=CC=C3S2,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])...",0
54759,54759,C1=CC=C2C(=C1)C(=CC(=N2)C3=CC=CS3)C(=O)N/N=C/C...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])...",0
...,...,...,...,...,...,...,...,...
623961,927247,CC1CCN(CC1)C(=O)COC2=CC3=C(C=C2)C4=C(C=C(C=C4)...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inconclusive,20.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])...",2
624036,927322,COC1=C(C=C(C=C1)C2=NN(C=C2C(=O)NC3=CC=CC(=C3)C...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inconclusive,20.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([8.]), tensor([6.])...",2
624131,927417,C1CCCN(CCC1)CC(=O)N/N=C/C2=CC=C(O2)[N+](=O)[O-],MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inconclusive,20.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])...",2
624172,927458,C1=CC(=CC=C1OCC(=O)NC2=CC3=C(C=C2)NN=C3)Cl,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inconclusive,20.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])...",2


In [20]:
balanced_data['PUBCHEM_ACTIVITY_OUTCOME'].value_counts()

PUBCHEM_ACTIVITY_OUTCOME
Inactive        8628
Active          8628
Inconclusive    8628
Name: count, dtype: int64

In [21]:
balanced_data['Sequence'].value_counts()

Sequence
MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKYSSRTEFRDKTDYIMYNPRPRDEPSSENPVSVSPLLCELAAARSRIHFNPTETTIGIVTCGGICPGLNDVIRSITLTGINVYNVKRVIGFRFGYWGLSKKGSQTAIELHRGRVTNIHHYGGTILGSSRGPQDPKEMVDTLERLGVNILFTVGGDGTQRGALVISQEAKRRGVDISVFGVPKTIDNDLSFSHRTFGFQTAVEKAVQAIRAAYAEAVSANYGVGVVKLMGRDSGFIAAQAAVASAQANICLVPENPISEQEVMSLLERRFCHSRSCVIIVAEGFGQDWGRGSGGYDASGNKKLIDIGVILTEKVKAFLKANKSRYPDSTVKYIDPSYMIRACPPSANDALFCATLATLAVHEAMAGATGCIIAMRHNNYILVPIKVATSVRRVLDLRGQLWRQVREITVDLGSDVRLARKLEIRRELEAINRNRDRLHEELAKL                13163
MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGMSVARMNFSHGSHEYHQTTINNVRQAAAELGVNIAIALDTKGPEIRTGQFVGGDAVMERGATCYVTTDPAFADKGTKDKFYIDYQNLSKVVRPGNYIYIDDGILILQVQSHEDEQTLECTVTNSHTISDRRGVNLPGCDVDLPAVSAKDRVDLQFGVEQGVDMIFASFIRSAEQVGDVRKALGPKGRDIMIICKIENHQGVQNIDSIIEESDGIMVARGDLGVEIPAEKVVVAQKILISKCNVAGKPVICATQMLESMTYNPRPTRAEVSDVANAVFNGADCVMLSGETAKGKYPNEVVQYMARICLEAQSALNEYVFFNSIKKLQHIPMSADEAVCSSAVNSVYETKAKAMVVLSNTGRSARLVAKYRPNCPIVCVTTRLQTCRQLNITQGVESVFFDADKLGHDEGKEHRVAAGVEFAKSKGYVQTGDYCVVIHA

In [22]:
##클레스 가중치
labels = balanced_data['ActivityLabel'].values
class_counts = np.bincount(labels)
total_samples = len(labels)
num_classes = len(class_counts)
class_weights = total_samples / (num_classes * class_counts)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

In [23]:
class_weights_tensor

tensor([1., 1., 1.], device='cuda:0')

In [24]:
df = balanced_data

In [25]:
df

,Unnamed: 0,PUBCHEM_EXT_DATASOURCE_SMILES,Sequence,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,ProteinGraph,MoleculeGraph,ActivityLabel
152188,152188,CN1C(=CC(=NS1(=O)=O)C2=CC=CS2)C(=O)NC3=CC=C(C=...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([7.]), tensor([6.])...",0
70469,70469,CC(=O)NC(CCS(=O)(=O)C)C(=O)NC1=CC=CC=C1C(=O)OC,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([8.])...",0
137845,137845,CN1C(=C(C(=N1)C(F)(F)F)C2=CC=CC=C2)NC(=O)CSCC(...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([7.]), tensor([6.])...",0
56640,56640,CC1=CC=CC=C1N(C(=O)C)/N=C\2/C(=O)C3=CC=CC=C3S2,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])...",0
54759,54759,C1=CC=C2C(=C1)C(=CC(=N2)C3=CC=CS3)C(=O)N/N=C/C...,MSQLAHNLTLSIFDPVANYRAARIICTIGPSTQSVEALKGLIQSGM...,Inactive,0.0,"[(x, [tensor([0.]), tensor([1.]), tensor([1.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])...",0
...,...,...,...,...,...,...,...,...
623961,927247,CC1CCN(CC1)C(=O)COC2=CC3=C(C=C2)C4=C(C=C(C=C4)...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inconclusive,20.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])...",2
624036,927322,COC1=C(C=C(C=C1)C2=NN(C=C2C(=O)NC3=CC=CC(=C3)C...,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inconclusive,20.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([8.]), tensor([6.])...",2
624131,927417,C1CCCN(CCC1)CC(=O)N/N=C/C2=CC=C(O2)[N+](=O)[O-],MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inconclusive,20.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])...",2
624172,927458,C1=CC(=CC=C1OCC(=O)NC2=CC3=C(C=C2)NN=C3)Cl,MAVESRSRVTSKLVKAHRAMLNSVTQEDLKVDRLPGADYPNPSKKY...,Inconclusive,20.0,"[(x, [tensor([0.]), tensor([0.]), tensor([0.])...","[(x, [tensor([6.]), tensor([6.]), tensor([6.])...",2


In [26]:
from sklearn.preprocessing import MinMaxScaler

# Calculate weights based on PUBCHEM_ACTIVITY_SCORE
scores = df['PUBCHEM_ACTIVITY_SCORE'].values

# Normalize weights to be between 0 and 1
scaler = MinMaxScaler()
weights = scaler.fit_transform(scores.reshape(-1, 1)).flatten()

# Add weights to the dataframe
df['Weight'] = weights

In [27]:
df['Weight'].value_counts()

Weight
0.000000    8628
0.219780    5421
0.450549    3326
0.461538    2129
0.230769    2046
0.439560     959
0.109890     814
0.472527     703
0.241758     251
0.483516     244
0.703297     185
0.923077     179
0.912088     169
0.934066     136
0.681319     131
0.692308     105
0.945055      82
0.714286      79
0.494505      63
0.956044      60
0.252747      60
0.505495      27
0.263736      23
0.967033      14
0.725275      11
0.736264      10
0.989011       8
0.274725       7
1.000000       5
0.285714       4
0.978022       3
0.307692       1
0.296703       1
Name: count, dtype: int64

In [28]:
#0을 없애기 아주 작은 값으로
weights = np.where(weights == 0, 1e-2, df['Weight'].values)
df['Weight'] = weights

In [29]:
df['Weight'].value_counts()

Weight
0.010000    8628
0.219780    5421
0.450549    3326
0.461538    2129
0.230769    2046
0.439560     959
0.109890     814
0.472527     703
0.241758     251
0.483516     244
0.703297     185
0.923077     179
0.912088     169
0.934066     136
0.681319     131
0.692308     105
0.945055      82
0.714286      79
0.494505      63
0.956044      60
0.252747      60
0.505495      27
0.263736      23
0.967033      14
0.725275      11
0.736264      10
0.989011       8
0.274725       7
1.000000       5
0.285714       4
0.978022       3
0.307692       1
0.296703       1
Name: count, dtype: int64

In [30]:

# Split the data #stratify -> 라벨 비율도 동일하게
#ActivityLabel 비율맞춰서 나누기 7:2:1
train_data, temp_data = train_test_split(df, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.33, random_state=42)

In [31]:
print(len(train_data),len(val_data),len(test_data))

18118 5203 2563


In [32]:
train_data['ActivityLabel'].value_counts()

ActivityLabel
1    6076
0    6021
2    6021
Name: count, dtype: int64

In [33]:
val_data['ActivityLabel'].value_counts()

ActivityLabel
0    1784
2    1715
1    1704
Name: count, dtype: int64

In [34]:
test_data['ActivityLabel'].value_counts()

ActivityLabel
2    892
1    848
0    823
Name: count, dtype: int64

In [35]:


def combine_graphs(mol_graph, protein_graph, weight, label):
    num_mol_nodes = mol_graph.num_nodes
    num_protein_nodes = protein_graph.num_nodes

    # 노드 특성 결합
    x = torch.cat([mol_graph.x, protein_graph.x], dim=0)

    # 에지 인덱스 결합
    protein_edge_index = protein_graph.edge_index + num_mol_nodes
    edge_index = torch.cat([mol_graph.edge_index, protein_edge_index], dim=1)

    # 상호작용 에지 추가
    interaction_edges = torch.tensor([[i, j + num_mol_nodes] for i in range(num_mol_nodes) for j in range(num_protein_nodes)], dtype=torch.long).t()
    interaction_edge_weight = torch.full((interaction_edges.size(1),), weight, dtype=torch.float)

    # 모든 에지 및 가중치 결합
    combined_edge_index = torch.cat([edge_index, interaction_edges], dim=1)
    combined_edge_weight = torch.cat([torch.ones(edge_index.size(1)), interaction_edge_weight], dim=0)

    # 데이터 객체 반환
    return Data(x=x, edge_index=combined_edge_index, edge_weight=combined_edge_weight, y=torch.tensor([label], dtype=torch.long))


# Function to process and combine rows
def process_and_combine(row):
    mol_graph = row['MoleculeGraph']
    protein_graph = row['ProteinGraph']
    weight = row['Weight']
    label = row['ActivityLabel']
    combined_graph = combine_graphs(mol_graph, protein_graph,weight, label)
    return combined_graph

# Apply the function to combine graphs
train_data['Graph'] = train_data.apply(process_and_combine, axis=1)
val_data['Graph'] = val_data.apply(process_and_combine, axis=1)
test_data['Graph'] = test_data.apply(process_and_combine, axis=1)





In [36]:
train_data['Graph'].values

array([Data(x=[529, 1], edge_index=[2, 15532], y=[1], edge_weight=[15532]),
       Data(x=[517, 1], edge_index=[2, 9518], y=[1], edge_weight=[9518]),
       Data(x=[506, 1], edge_index=[2, 9781], y=[1], edge_weight=[9781]),
       ...,
       Data(x=[528, 1], edge_index=[2, 15031], y=[1], edge_weight=[15031]),
       Data(x=[507, 1], edge_index=[2, 10266], y=[1], edge_weight=[10266]),
       Data(x=[508, 1], edge_index=[2, 10757], y=[1], edge_weight=[10757])],
      dtype=object)

In [37]:
val_data['Graph'].values

array([Data(x=[521, 1], edge_index=[2, 17118], y=[1], edge_weight=[17118]),
       Data(x=[530, 1], edge_index=[2, 16033], y=[1], edge_weight=[16033]),
       Data(x=[521, 1], edge_index=[2, 11522], y=[1], edge_weight=[11522]),
       ...,
       Data(x=[508, 1], edge_index=[2, 10759], y=[1], edge_weight=[10759]),
       Data(x=[511, 1], edge_index=[2, 12226], y=[1], edge_weight=[12226]),
       Data(x=[520, 1], edge_index=[2, 11023], y=[1], edge_weight=[11023])],
      dtype=object)

In [38]:
test_data['Graph'].values

array([Data(x=[524, 1], edge_index=[2, 13027], y=[1], edge_weight=[13027]),
       Data(x=[519, 1], edge_index=[2, 10524], y=[1], edge_weight=[10524]),
       Data(x=[514, 1], edge_index=[2, 13691], y=[1], edge_weight=[13691]),
       ...,
       Data(x=[516, 1], edge_index=[2, 9019], y=[1], edge_weight=[9019]),
       Data(x=[509, 1], edge_index=[2, 11248], y=[1], edge_weight=[11248]),
       Data(x=[523, 1], edge_index=[2, 12526], y=[1], edge_weight=[12526])],
      dtype=object)

In [39]:
print(len(train_data['Graph']),len(val_data['Graph']),len(test_data['Graph']))

18118 5203 2563


In [40]:

# Create DataLoader for each split
train_loader = DataLoader(train_data['Graph'].tolist(), batch_size=64, shuffle=True)
val_loader = DataLoader(val_data['Graph'].tolist(), batch_size=64, shuffle=False)
test_loader = DataLoader(test_data['Graph'].tolist(), batch_size=64, shuffle=False)

In [41]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[33039, 1], edge_index=[2, 739409], y=[64], edge_weight=[739409], batch=[33039], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[33060, 1], edge_index=[2, 761064], y=[64], edge_weight=[761064], batch=[33060], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[33161, 1], edge_index=[2, 799923], y=[64], edge_weight=[799923], batch=[33161], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[33106, 1], edge_index=[2, 794908], y=[64], edge_weight=[794908], batch=[33106], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[33108, 1], edge_index=[2, 806996], y=[64], edge_weight=[806996], batch=[33108], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[32983, 1], edge_index=[2, 762433], y=[64], edge_weight=[762433], batch=[32983], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(x=[33042, 1], ed

In [42]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super(GCN, self).__init__()
        self.convs = torch.nn.ModuleList()
        
        # 첫 번째 GCNConv 층 추가
        self.convs.append(GCNConv(in_channels, hidden_channels))
        
        # 중간 GCNConv 층들 추가
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        # 마지막 GCNConv 층 추가
        self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        self.lin = Linear(hidden_channels, out_channels)  # 3-class classification

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            
        
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x

In [52]:
def train(model, optimizer, train_loader, device, criterion, epoch, num_epochs):
    model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for batch in train_loader:
            batch_x = batch.x.to(device)
            batch_edge_index = batch.edge_index.to(device)
            batch_y = batch.y.to(device)
            batch_batch = batch.batch.to(device)

            optimizer.zero_grad()
            
            # 모델 출력 계산
            out = model(batch_x, batch_edge_index, batch_batch)
            
            # 손실 계산
            loss = criterion(out, batch_y) 
            loss.backward()


            optimizer.step()
            
            train_loss += loss.item()
            
            # 예측값 계산 (소프트맥스 사용 없이)
            predicted = torch.argmax(out, 1)
            
            correct_train += (predicted == batch_y).sum().item()
            total_train += batch_y.size(0)
            
            pbar.set_postfix({'loss': train_loss / (pbar.n + 1), 'accuracy': correct_train / total_train})
            pbar.update(1)

    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = correct_train / total_train

    return avg_train_loss, train_accuracy

In [53]:
def evaluate(model, val_loader, device, criterion):
    model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        with tqdm(total=len(val_loader), desc='Validation', unit='batch') as pbar:
            for batch in val_loader:
                batch_x = batch.x.to(device)
                batch_edge_index = batch.edge_index.to(device)
                batch_y = batch.y.to(device)
                batch_batch = batch.batch.to(device)

                out = model(batch_x, batch_edge_index, batch_batch)
                    
                # 손실 계산
                loss = criterion(out, batch_y) 
                val_loss += loss.item()
                
                # 예측값 계산 (소프트맥스 사용 없이)
                predicted = torch.argmax(out, 1)
                
                # 정확도 계산
                correct_val += (predicted == batch_y).sum().item()
                total_val += batch_y.size(0)
                    
                # 모든 예측 및 라벨 저장
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())
                all_probs.extend(out.cpu().numpy())
                
                pbar.update(1)
        
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct_val / total_val

    return avg_val_loss, val_accuracy


In [54]:

def evaluate_test(model, test_loader, device, criterion):
    model.eval()
    test_loss = 0
    correct_test = 0
    total_test = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        with tqdm(total=len(test_loader), desc='Testing', unit='batch') as pbar:
            for batch in test_loader:
                batch_x = batch.x.to(device)
                batch_edge_index = batch.edge_index.to(device)
                batch_y = batch.y.to(device)
                batch_batch = batch.batch.to(device)

                out = model(batch_x, batch_edge_index, batch_batch)
                
                # 손실 계산
                loss = criterion(out, batch_y) 
                test_loss += loss.item()
                
                # 예측값 계산 (소프트맥스 사용 없이)
                predicted = torch.argmax(out, 1)
                
                # 정확도 계산
                correct_test += (predicted == batch_y).sum().item()
                total_test += batch_y.size(0)
                
                # 모든 예측값, 실제 라벨, 확률 값을 리스트에 저장
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())
                all_probs.extend(F.softmax(out, dim=1).cpu().numpy())  # 확률 계산은 여기서 수행
                
                pbar.update(1)
    
    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = correct_test / total_test

    # 추가적인 평가 지표 계산
    test_f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)
    test_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
    test_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
    test_auc = roc_auc_score(all_labels, np.array(all_probs), multi_class='ovo')

    # 결과 출력
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}, Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f}, Test AUC: {test_auc:.4f}')

    return avg_test_loss, test_accuracy, test_f1, test_precision, test_recall, test_auc



예측 부분 조금 다르게 함

In [43]:
def train(model, optimizer, train_loader, device, criterion, epoch, num_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for batch in train_loader:
            # 배치 데이터를 각 변수로 분리하고, device로 이동
            batch_x = batch.x.to(device)  # 노드 특성
            batch_edge_index = batch.edge_index.to(device)  # 엣지 정보
            batch_y = batch.y.to(device)  # 레이블
            batch_batch = batch.batch.to(device)  # 배치 정보

            optimizer.zero_grad()  # 그래디언트를 초기화
            out = model(batch_x, batch_edge_index, batch_batch)  # 모델의 출력을 계산
            loss = criterion(out, batch_y)  # 손실을 계산
            
            loss.backward()  # 역전파를 통해 그래디언트를 계산
            optimizer.step()  # 옵티마이저를 사용하여 모델 파라미터를 업데이트
            
            # 배치 크기로 스케일링된 손실을 총 손실에 더gkrl
            train_loss += loss.item() * batch_y.size(0)
            _, predicted = torch.max(out, 1)  # 예측값을 계산
            correct_train += (predicted == batch_y).sum().item()  # 맞은 예측의 개수를 누적
            total_train += batch_y.size(0)  # 총 샘플 수를 누적
            
            # print(f'Batch y: {batch.y}')

            # 진행 상황을 업데이트
            pbar.set_postfix({'loss': train_loss / total_train, 'accuracy': correct_train / total_train})
            pbar.update(1)

    # 평균 손실과 정확도를 계산.
    avg_train_loss = train_loss / total_train
    train_accuracy = correct_train / total_train

    return avg_train_loss, train_accuracy  

In [44]:

def evaluate(model, val_loader, device, criterion):
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():  # 그래디언트를 계산하지 않도록 설정
        with tqdm(total=len(val_loader), desc='Validation', unit='batch') as pbar:
            for batch in val_loader:
                # 배치 데이터를 각 변수로 분리하고, device로 이동
                batch_x = batch.x.to(device)  # 노드 특성
                batch_edge_index = batch.edge_index.to(device)  # 엣지 정보
                batch_y = batch.y.to(device)  # 레이블
                batch_batch = batch.batch.to(device)  # 배치 정보

                out = model(batch_x, batch_edge_index, batch_batch)  # 모델의 출력을 계산
                loss = criterion(out, batch_y)  # 손실을 계산
                
                val_loss += loss.item() * batch_y.size(0)  # 배치 크기로 스케일링된 손실을 총 손실에 더하기
                
                _, predicted = torch.max(out, 1)  # 예측값을 계산
                correct_val += (predicted == batch_y).sum().item()  # 맞은 예측의 개수를 누적
                total_val += batch_y.size(0)  # 총 샘플 수를 누적
                all_preds.extend(predicted.cpu().numpy())  # 예측값을 리스트에 저장
                all_labels.extend(batch_y.cpu().numpy())  # 실제 라벨을 리스트에 저장
                all_probs.extend(F.softmax(out, dim=1).cpu().numpy())  # 예측 확률을 리스트에 저장
                
                # print(f'Batch y: {batch.y}')

                pbar.update(1)  # 진행 상황을 업데이트
    
    avg_val_loss = val_loss / total_val  
    val_accuracy = correct_val / total_val  

    return avg_val_loss, val_accuracy 

In [45]:

def evaluate_test(model, test_loader, device, criterion):
    model.eval()
    test_loss = 0.0
    correct_test = 0
    total_test = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():  # 그래디언트를 계산하지 않도록 설정
        with tqdm(total=len(test_loader), desc='Testing', unit='batch') as pbar:
            for batch in test_loader:
                # 배치 데이터를 각 변수로 분리하고, device로 이동
                batch_x = batch.x.to(device)  # 노드 특성
                batch_edge_index = batch.edge_index.to(device)  # 엣지 정보
                batch_y = batch.y.to(device)  # 레이블
                batch_batch = batch.batch.to(device)  # 배치 정보

                out = model(batch_x, batch_edge_index, batch_batch)  # 모델의 출력을 계산
                loss = criterion(out, batch_y)  # 손실을 계산
                
                test_loss += loss.item() * batch_y.size(0)  # 배치 크기로 스케일링된 손실을 총 손실에 더하기
                
                _, predicted = torch.max(out, 1)  # 예측값을 계산
                correct_test += (predicted == batch_y).sum().item()  # 맞은 예측의 개수를 누적
                total_test += batch_y.size(0)  # 총 샘플 수를 누적합니다.
                all_preds.extend(predicted.cpu().numpy())  # 예측값을 리스트에 저장
                all_labels.extend(batch_y.cpu().numpy())  # 실제 라벨을 리스트에 저장
                all_probs.extend(F.softmax(out, dim=1).cpu().numpy())  # 예측 확률을 리스트에 저장
                
                pbar.update(1)  # 진행 상황을 업데이트
    
    avg_test_loss = test_loss / total_test  
    test_accuracy = correct_test / total_test  

    # 추가적인 평가 지표 계산
    test_f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)
    test_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
    test_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
    test_auc = roc_auc_score(all_labels, np.array(all_probs), multi_class='ovo')

    # 결과 출력
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}, Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f}, Test AUC: {test_auc:.4f}')

    return avg_test_loss, test_accuracy, test_f1, test_precision, test_recall, test_auc  


In [46]:
import torch.optim as optim

In [62]:

# Training the model and tracking history
model = GCN(in_channels=1, hidden_channels=128, out_channels=3, num_layers=4).to(device)
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

history = {
    'epoch': [],
    'train_loss': [],
    'val_loss': [],
    'train_accuracy': [],
    'val_accuracy': [],
}

In [ ]:
# Training loop
num_epochs = 100  # Reduce number of epochs for debugging purposes

for epoch in range(num_epochs):
    avg_train_loss, train_accuracy = train(model, optimizer, train_loader, device, criterion, epoch, num_epochs)
    avg_val_loss, val_accuracy = evaluate(model, val_loader, device, criterion)
    
    history['epoch'].append(epoch + 1)
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(avg_val_loss)
    history['train_accuracy'].append(train_accuracy)
    history['val_accuracy'].append(val_accuracy)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
    print(f'Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    
    scheduler.step(avg_val_loss)
    print(f'Learning Rate: {scheduler.get_last_lr()[0]:.6f}')

In [59]:
# Evaluate on test data and store results in history
test_loss, test_accuracy, test_f1, test_precision, test_recall, test_auc = evaluate_test(model, test_loader, device, criterion)

history['test_loss'] = test_loss
history['test_accuracy'] = test_accuracy
history['test_f1'] = test_f1
history['test_precision'] = test_precision
history['test_recall'] = test_recall
history['test_auc'] = test_auc

print(f'Test Results - Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, F1: {test_f1:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, AUC: {test_auc:.4f}')


Testing: 100%|██████████| 41/41 [00:00<00:00, 43.07batch/s]

Test Loss: 1.0987, Test Accuracy: 0.3309, Test F1 Score: 0.1645, Test Precision: 0.7786, Test Recall: 0.3309, Test AUC: 0.5000
Test Results - Loss: 1.0987, Accuracy: 0.3309, F1: 0.1645, Precision: 0.7786, Recall: 0.3309, AUC: 0.5000


In [60]:
history_save_path = 'e100_history1_2_64-128-4.json'
with open(history_save_path, 'w') as f:
    json.dump(history, f)
print(f'Training history saved to {history_save_path}')


Training history saved to result\gcn1_1\e100_history1_2_64-128-2.json


In [ ]:

# 학습 및 검증 손실 시각화
plt.figure(figsize=(10, 5))
plt.plot(history['epoch'], history['train_loss'], label='Train Loss')
plt.plot(history['epoch'], history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('e100_gcn_1_2_64-128-4 Train and Validation Loss')
plt.legend()
plt.grid(True)
plt.savefig('e100_train1_2_64-128-4.png')
plt.show()

# 학습 및 검증 정확도 시각화
plt.figure(figsize=(10, 5))
plt.plot(history['epoch'], history['train_accuracy'], label='Train Accuracy')
plt.plot(history['epoch'], history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('e100_gcn_1_2_64-128-4 Train and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.savefig('e100_val_1_2_64-128-4.png')
plt.show()
